### Load Envs 

In [1]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI
from tools.get_tools import get_all_schemas_and_tools_by_default
from langchain import hub
from utils.load_envs import Environ

from chains.qa_evaluators import CustomQAEvaluator



GPT_VER = '1106'
Environ(GPT_VER)

### ReAct

In [ ]:
#### ReAct Agent
from agents.react_agents import ReActAgent
react_prompt = "hwchase17/react-multi-input-json"

react_agent = ReActAgent(reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt=react_prompt, 
                        schemas_and_tools=get_all_schemas_and_tools_by_default(azure_gpt_version=GPT_VER), 
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version=GPT_VER)), 
                        stop_words=["Observation"], 
                        thought_word= 'Thought', 
                        action_word='Action',
                        file_logging=True,
                        horizon=5)


react_agent.clear_logs()
react_agent.run_agent_trials(num_trials=3, 
query="Which one is bigger, 'World War 2 outbreak year plus Donald Trumps birth month' vs. 'U.S. Independece-day year plus Michael Jordan birth year'?",
# query="Sort the following list in an descending order [1, 6, 3, 6, 8, 3, 5, 0]",
# query="How is the weather in Boston tomorrow?",
# reference="'U.S. Independece-day year plus Michael Jordan birth year' is bigger.")
)


### Reflexion ReAct

In [ ]:
#### Reflexion Agent
from agents.reflexion_react_agents import ReflexionReActAgent
from chains.reflexion_chains import BaseReflexionChain
from fewshot_examples.reflections import REACT_REFLECTION_HEADER, REACT_REFLECTIONS
   
reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/experimental-reflextion", 
                    schemas_and_tools=get_all_schemas_and_tools_by_default(azure_gpt_version=GPT_VER),
                    reflexion_examples=REACT_REFLECTIONS
                    )

reflexion_prompt = "jet-taekyo-lee/reflexion-react"
reflexion_agent = ReflexionReActAgent(
                        reflexion_chain = reflexion_chains,
                        reflexion_header = REACT_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt=react_prompt, 
                        schemas_and_tools=get_all_schemas_and_tools_by_default(azure_gpt_version=GPT_VER), 
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version=GPT_VER)), 
                        stop_words=["Observation"], 
                        thought_word= 'Thought', 
                        action_word='Action',
                        file_logging=True,
                        horizon=5)



reflexion_agent.clear_logs()
reflexion_agent.run_agent_trials(num_trials=3, 
query="Which one is bigger, 'World War 2 outbreak year plus Donald Trumps birth month' vs. 'U.S. Independece-day year plus Michael Jordan birth year'? And How is the weather in Boston tomorrow?",
reference="'U.S. Independece-day year plus Michael Jordan birth year' is bigger.")



### OpenAI FunctionCalling agent

In [4]:
# #### OepnAI Agent
from agents.openai_function_calling_agent import OpenAIFuntionCallingAgent
openai_agent = OpenAIFuntionCallingAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = "jet-taekyo-lee/openai-function-calling-agent",
    schemas_and_tools=get_all_schemas_and_tools_by_default(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI(version=GPT_VER)),
    action_word = 'Invoking',
    horizon = 3)

openai_agent.clear_logs()
openai_agent.run_agent_trials(num_trials=3, 
query="How old are the current Korean president and Joe Biden?",
reference="Joe Biden is 81 years old and the current Korean president, Yoon Suk Yeol, is 62 years old.")

[INFO] Query: How old are the current Korean president and Joe Biden?
[INFO] Trial 1
[INFO] Invoking 1: YDCSearch(query='current age of the president of South Korea')
[INFO] Observation 1: Yoon Suk Yeol, the current president of South Korea, was born on December 18, 1960. As of today, he is 62 years old.
[INFO] Invoking 2: YDCSearch(query='current age of Joe Biden')


RateLimitError for _get_function_observation. -----> Will automatically retry 54 seconds later.
54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] Observation 2: As of the current date, Joe Biden is 80 years old. He was born on November 20, 1942.
[INFO] Invoking 3: Yoon Suk Yeol is 62 years old and Joe Biden is 80 years old.
[INFO] Answer: Yoon Suk Yeol is 62 years old and Joe Biden is 80 years old.
[INFO] Jugdement: Your answer is incorrect. The correct answer is "Joe Biden is 81 years old and the current Korean president, Yoon Suk Yeol, is 62 years old."
[INFO] Trial 2
[INFO] Invoking 1: YDCSearch(query='current age of President of South Korea')


RateLimitError for _get_function_observation. -----> Will automatically retry 52 seconds later.
52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] Observation 1: As of the information provided, Yoon Suk Yeol, the current President of South Korea, was born on December 18, 1960. Therefore, as of today's date, April 28, 2023, he is 62 years old.
[INFO] Invoking 2: YDCSearch(query='current age of Joe Biden')


...

[INFO] Observation 2: As of the current date, Joe Biden is 80 years old. He was born on November 20, 1942.
[INFO] Invoking 3: The current Korean president, Yoon Suk Yeol, is 62 years old, and Joe Biden is 80 years old.
[INFO] Answer: The current Korean president, Yoon Suk Yeol, is 62 years old, and Joe Biden is 80 years old.
[INFO] Jugdement: Your answer is incorrect. The correct answer is "Joe Biden is 81 years old and the current Korean president, Yoon Suk Yeol, is 62 years old."
[INFO] Trial 3
[INFO] Invoking 1: YDCSearch(query='current Korean president age')


RateLimitError for _get_function_observation. -----> Will automatically retry 53 seconds later.
53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] Observation 1: The current president of South Korea, Yoon Suk Yeol, was born on December 18, 1960. As of today, he is 61 years old.
[INFO] Invoking 2: YDCSearch(query='Joe Biden age')


RateLimitError for _get_function_observation. -----> Will automatically retry 55 seconds later.
55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] Observation 2: Joe Biden is 81 years old. He was born on November 20, 1942.
[INFO] Invoking 3: Yoon Suk Yeol: 61 Jahre alt, Joe Biden: 81 Jahre alt.
[INFO] Answer: Yoon Suk Yeol: 61 Jahre alt, Joe Biden: 81 Jahre alt.
[INFO] Jugdement: Your answer is incorrect. The correct answer is "Joe Biden is 81 years old and the current Korean president, Yoon Suk Yeol, is 62 years old."


### Reflexion OpenAI FunctionCalling agent

In [2]:
# # #### Reflexion OepnAI Agent
from agents.reflexion_openai_function_calling_agent import ReflexionOpenAIFuntionCallingAgent
from chains.reflexion_chains import BaseReflexionChain
from fewshot_examples.reflections import OPENAI_REFLECTIONS, REACT_REFLECTION_HEADER
   
openai_reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/openai-reflexion-chain-prompt", 
                    schemas_and_tools=get_all_schemas_and_tools_by_default(azure_gpt_version=GPT_VER),
                    reflexion_examples=OPENAI_REFLECTIONS
                    )

openai_reflexion_prompt = "jet-taekyo-lee/rexlexion-openai-function-calling-agent"
openai_reflexion_agent = ReflexionOpenAIFuntionCallingAgent(
                        reflexion_chain = openai_reflexion_chains,
                        reflexion_header = REACT_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt=openai_reflexion_prompt, 
                        schemas_and_tools=get_all_schemas_and_tools_by_default(azure_gpt_version=GPT_VER),
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version=GPT_VER)),
                        action_word='Invoking',
                        file_logging=True,
                        horizon = 3)

openai_reflexion_agent.clear_logs()
openai_reflexion_agent.run_agent_trials(num_trials=3, 
query="How old are the current Korean president and Joe Biden?",
reference="Joe Biden is 81 years old and the current Korean president, Yoon Suk Yeol, is 62 years old.")


[INFO] Query: How old are the current Korean president and Joe Biden?
[INFO] Trial 1
[INFO] Invoking 1: YDCSearch(query='current Korean president age')
[INFO] Observation 1: The current President of South Korea, Yoon Suk Yeol, was born on December 18, 1960. As of the current date, October 26, 2022, he is 61 years old.
[INFO] Invoking 2: YDCSearch(query='Joe Biden age')


RateLimitError for _get_function_observation. -----> Will automatically retry 57 seconds later.
57 56 55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] Observation 2: Joe Biden is 81 years old. He was born on November 20, 1942.
[INFO] Invoking 3: Yoon Suk Yeol is 61 years old and Joe Biden is 81 years old.
[INFO] Answer: Yoon Suk Yeol is 61 years old and Joe Biden is 81 years old.
[INFO] Jugdement: Your answer is incorrect. The correct answer is "Joe Biden is 81 years old and the current Korean president, Yoon Suk Yeol, is 62 years old."
[INFO] Trial 2
[INFO] Reflexion......
[INFO] You have attempted to answer following question before and failed. The following reflection(s) give a plan to avoid failing to answer the question in the same way you did previously. Use them to improve your strategy of correctly answering the given question.
- The query asked for the ages of two individuals, the current Korean president and Joe Biden. The initial steps taken to search for their ages separately were correct. However, I did not verify the accuracy of the information obtained from the search results. For Joe Biden, the search result st

RateLimitError for _invoke_agent_action. -----> Will automatically retry 47 seconds later.
47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] Invoking 2: YDCSearch(query='current Korean president 2023')
[INFO] Observation 2: The current president of South Korea as of 2023 is Yoon Suk Yeol.
[INFO] Invoking 3: YDCSearch(query='Yoon Suk Yeol birthdate')


...

[INFO] Observation 3: Yoon Suk Yeol was born on 18 December 1960.
[INFO] Jugdement: You failed to provide an answer because you exceeded the permitted number of reasoning steps. You must give an answer within 3 steps.
[INFO] Trial 3
[INFO] Reflexion......
[INFO] You have attempted to answer following question before and failed. The following reflection(s) give a plan to avoid failing to answer the question in the same way you did previously. Use them to improve your strategy of correctly answering the given question.
- The query asked for the ages of two individuals, the current Korean president and Joe Biden. The initial steps taken to search for their ages separately were correct. However, I did not verify the accuracy of the information obtained from the search results. For Joe Biden, the search result stated that he is 81 years old, which is incorrect as of the current date provided (October 26, 2022). Joe Biden, born in 1942, would be 79 years old, not 81. This mistake led to an i

RateLimitError for _get_function_observation. -----> Will automatically retry 47 seconds later.
47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] Observation 1: The current president of South Korea, Yoon Suk Yeol, was born on 18 December 1960.
[INFO] Invoking 2: YDCSearch(query='Joe Biden birthdate')


RateLimitError for _get_function_observation. -----> Will automatically retry 54 seconds later.
54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] Observation 2: Joe Biden was born on November 20, 1942.
[INFO] Invoking 3: GetDatetime(IANA_timezone='Asia/Seoul')
[INFO] Observation 3: timezone: Asia/Seoul, current time(year-month-day-hour-minute): 2023-11-28-20-6
[INFO] Jugdement: You failed to provide an answer because you exceeded the permitted number of reasoning steps. You must give an answer within 3 steps.


### Parallel Function Calling agent

In [ ]:
#### Parallel Function Calling Agent
from agents.parallel_func_calling_agent import OpenAIParallelFuntionCallingAgent
parallel_function_calling_agent = OpenAIParallelFuntionCallingAgent(
    reasoninig_engine = QuickAzureChatOpenAI(),
    base_prompt = 'jet-taekyo-lee/parallel-function-calling-agent',
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI()),
    action_word = 'Invoking',
    schemas_and_tools = get_all_schemas_and_tools_by_default(llm=QuickAzureChatOpenAI())
)



parallel_function_calling_agent.clear_logs()
parallel_function_calling_agent.run_agent_trials(num_trials=3, 
query="How old are the current Korean president and Joe Biden?",
reference="Joe Biden is 81 years old and the current Korean president, Yoon Suk Yeol, is 62 years old.")

